In this quickstart we'll show you how to:

* Get setup with LangChain, LangSmith and LangServe
* Use the most basic and common components of LangChain: prompt templates, models, and output parsers
* Use LangChain Expression Language, the protocol that LangChain is built on and which facilitates component chaining
* Build a simple application with LangChain
* Trace your application with LangSmith
* Serve your application with LangServe


# Setup

## Installation

To install LangChain run:

# Building with LangChain

LangChain enables building application that connect external sources of data and computation to LLMs. In this quickstart, we will walk through a few different ways of doing that. We will start with a simple LLM chain, which just relies on information in the prompt template to respond. Next, we will build a retrieval chain, which fetches data from a separate database and passes that into the prompt template. We will then add in chat history, to create a conversation retrieval chain. This allows you to interact in a chat manner with this LLM, so it remembers previous questions. Finally, we will build an agent - which utilizes an LLM to determine whether or not it needs to fetch data to answer questions. We will cover these at a high level, but there are lot of details to all of these! We will link to relevant docs.


In [1]:
pip install langchain

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [32]:
# 连接本地部署的 Ollama，并使用 Llama2 模型
# PS：需要现在本机部署好 Llama2 / Llama3

from langchain_community.llms import Ollama
llm = Ollama(model="llama2")

API Reference:
* [Ollama](https://api.python.langchain.com/en/latest/llms/langchain_community.llms.ollama.Ollama.html)

In [33]:
# 直接给LLM发送问题，不做任何prompt，embedding和context

llm.invoke("how can langsmith help with testing?")

'Langsmith is a tool that can assist with testing by providing features such as:\n\n1. Code Analysis: Langsmith can analyze your codebase and identify potential issues, such as vulnerabilities, performance bottlenecks, and security risks.\n2. Automated Testing: Langsmith can automatically test your codebase using a variety of testing frameworks and libraries, including Selenium, JUnit, and Pytest.\n3. Code Refactoring: Langsmith can suggest improvements to your codebase, such as renaming variables, consolidating redundant code, and improving code organization.\n4. Dependency Management: Langsmith can help you manage dependencies between different parts of your codebase, ensuring that they are consistent and up-to-date.\n5. Collaboration Tools: Langsmith provides tools for collaboration, such as real-time commenting and feedback, and the ability to assign tasks and track progress.\n6. Integration with CI/CD Pipelines: Langsmith can be integrated with your Continuous Integration/Continuo

In [34]:
# 引入聊天对话的 Prompt Template，让LLM更加聚焦

from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world class technical documentation writer."),
    ("user", "{input}")
])

API Reference:
* [ChatPromptTemplate](https://api.python.langchain.com/en/latest/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html)


In [35]:
# 将 Chat Prompt Template和LLM做一个Chain关联

chain = prompt | llm

In [36]:
# 带上Chat Prompt Template 再次给LLM发送问题，此时LLM仍然不知道问题的答案，但能够用更加合适的
# 语气和措辞（as a world class technical documentation writer）来回答

chain.invoke({"input": "how can langsmith help with testing?"})

"\nAs a world-class technical documentation writer, I must say that Langsmith is a powerful tool that can greatly assist in the testing process. Here are some ways Langsmith can help:\n\n1. Automated Testing: Langsmith's AI-powered engine can automatically generate test cases based on your technical documentation. This can save you a tremendous amount of time and effort compared to writing test cases manually. With Langsmith, you can create comprehensive test suites that cover all aspects of your product, including functionality, performance, security, and usability.\n2. Test Case Prioritization: Langsmith's AI algorithms can prioritize test cases based on risk and complexity. This ensures that the most critical tests are run first, reducing the overall testing time and effort. By automating the test case prioritization process, you can free up your testing resources for more complex and higher-risk tests.\n3. Defect Prediction: Langsmith's AI models can analyze your technical document

In [37]:
# 将LLM返回结果为一个String方便后续处理

from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

API Reference:
* [StrOutputParser](https://api.python.langchain.com/en/latest/output_parsers/langchain_core.output_parsers.string.StrOutputParser.html)

In [8]:
chain = prompt | llm | output_parser

In [9]:
chain.invoke({"input": "how can langsmith help with testing?"})

"\nAs a world-class technical documentation writer, I must say that Langsmith is a versatile tool that can greatly assist with testing. Here are some ways in which Langsmith can help:\n\n1. Automated Documentation Testing: Langsmith allows you to create automated tests for your technical documents using AI-powered language analysis. This helps ensure that your documentation is accurate, consistent, and up-to-date, which is crucial for effective testing.\n2. Customizable Test Cases: With Langsmith, you can create custom test cases tailored to your specific documentation needs. This enables you to thoroughly test your documents and identify any potential issues before they become problems.\n3. Integration with Test Management Tools: Langsmith integrates seamlessly with popular test management tools like JIRA, TestRail, and PractiTest. This integration allows you to easily manage and organize your testing process, streamlining the entire testing workflow.\n4. Intelligent Review and Feedba

## Diving Deeper

至此，已经成功的通过 LangChain python Lib代码拼凑一个基本的LLM Chain，这里只涉及到了最基本的prompts，模型和输出格式parsers。

这里没有涉及到：

* Chuning(text splitting, 业界英文环境普遍使用Ada_200模型)
* Embedding
* Indexing
* Vector Store
* Retrival

等RAG中每个环节需要涉及到的技术细节。

[For a deeper dive](https://python.langchain.com/docs/modules/model_io/)

# Retrieval Chain

To properly answer the original question ("how can langsmith help with testing?"), we need to provide additional context to the LLM. We can do this via `retrieval`. Retrieval is useful when you have **too much data** to pass to the LLM directly. You can then use a retriever to fetch only the most relevant pieces and pass those in.

In this process, we will look up relevant documents from a Retriever and then pass them into the prompt. A Retriever can be backed by anything - a SQL table, the internet, etc - but in this instance we will populate a vector store and use that as a retriever. For more information on vectorstores, see [this documentation](https://python.langchain.com/docs/modules/data_connection/vectorstores/).

First, we need to load the data that we want to index. To do this, we will use the WebBaseLoader. This requires installing [BeautifulSoup:](https://beautiful-soup-4.readthedocs.io/en/latest/)



In [12]:
pip install beautifulsoup4

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


After that, we can import and use WebBaseLoader.

In [38]:
# 从互联网Load一篇文章

from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")

docs = loader.load()

API Reference:
 * [WebBaseLoader](https://api.python.langchain.com/en/latest/document_loaders/langchain_community.document_loaders.web_base.WebBaseLoader.html)

Next, we need to index it into a vectorstore. This requires a few components, namely an [embedding model](https://python.langchain.com/docs/modules/data_connection/text_embedding/) and a [vectorstore](https://python.langchain.com/docs/modules/data_connection/vectorstores/).

For embedding models, we once again provide examples for accessing via API or by running local models.

In [39]:
# 这里使用本地部署的Ollama的embedding接口

from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings()

API Reference:
* [OllamaEmbeddings](https://api.python.langchain.com/en/latest/embeddings/langchain_community.embeddings.ollama.OllamaEmbeddings.html)

Now, we can use this embedding model to ingest documents into a vectorstore. We will use a simple local vectorstore, [FAISS](https://python.langchain.com/docs/integrations/vectorstores/faiss/), for simplicity's sake.

First we need to install the required packages for that:

在将 Embedding后的向量写入向量库前，为简单起见，入FAISS做本地内存向量存储


In [41]:
pip install faiss-cpu

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


Then we can build our index:

In [42]:
# 对刚才的文章进行 embedding（向量化），然后写入 本地 FAISS 向量库中。

from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

API Reference:
* [FAISS](https://api.python.langchain.com/en/latest/vectorstores/langchain_community.vectorstores.faiss.FAISS.html)
* [RecursiveCharacterTextSplitter](https://api.python.langchain.com/en/latest/character/langchain_text_splitters.character.RecursiveCharacterTextSplitter.html)

Now that we have this data indexed in a vectorstore, we will create a retrieval chain. This chain will take an incoming question, look up relevant documents, then pass those documents along with the original question into an LLM and ask it to answer the original question.

First, let's set up the chain that takes a question and the retrieved documents and generates an answer.

In [43]:
# 有了原始文章的向量索引后，就可以拼凑一个问答式的 Prompt，并结合已经建好的向量索引
# 根据问题本身的向量索引和文档向量索引的距离进行向量搜索

from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)


API Reference:
* [create_stuff_documents_chain](https://api.python.langchain.com/en/latest/chains/langchain.chains.combine_documents.stuff.create_stuff_documents_chain.html)

If we wanted to, we could run this ourselves by passing in documents directly:

In [44]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})

'Based on the provided context, Langsmith can help with testing by allowing users to visualize their test results.'

API Reference:
* [Document](https://api.python.langchain.com/en/latest/documents/langchain_core.documents.base.Document.html)

However, we want the documents to first come from the retriever we just set up. That way, we can use the retriever to dynamically select the most relevant documents and pass those in for a given question.

In [20]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

API Reference:
* [create_retrieval_chain](https://api.python.langchain.com/en/latest/chains/langchain.chains.retrieval.create_retrieval_chain.html)

We can now invoke this chain. This returns a dictionary - the response from the LLM is in the answer key

In [21]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

# LangSmith offers several features that can help with testing:...


Based on the provided context, LangSmith can help with testing in several ways:

1. Creation of test cases: LangSmith allows developers to create datasets, which are collections of inputs and reference outputs, that can be used to run tests on their LLM applications.
2. Annotation of traces: LangSmith supports sending runs to annotation queues, where annotators can closely inspect interesting traces and annotate them with respect to different criteria. This helps in catching regressions across important evaluation criteria.
3. Monitoring and A/B testing: LangSmith provides monitoring charts that allow users to track key metrics over time. Users can expand to view metrics for a given period and drill down into a specific data point to get a trace table for that time period, which is helpful for debugging production issues.
4. Automations: LangSmith allows users to perform actions on traces in near real-time, such as automatically scoring traces, sending them to annotation queues, or se

This answer should be much more accurate!

## Diving Deeper
We've now successfully set up a basic retrieval chain. We only touched on the basics of retrieval - for a deeper dive into everything mentioned here, see [this section of documentation](https://python.langchain.com/docs/modules/data_connection/).



# Conversation Retrieval Chain

The chain we've created so far can only answer single questions. One of the main types of LLM applications that people are building are chat bots. So how do we turn this chain into one that can answer follow up questions?

We can still use the `create_retrieval_chain` function, but we need to change two things:

1. The retrieval method should now not just work on the most recent input, but rather should take the whole history into account.
1. The final LLM chain should likewise take the whole history into account

**Updating Retrieval**

In order to update retrieval, we will create a new chain. This chain will take in the most recent input (`input`) and the conversation history (`chat_history`) and use an LLM to generate a search query.

In [24]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

API Reference:
 * [create_history_aware_retriever](https://api.python.langchain.com/en/latest/chains/langchain.chains.history_aware_retriever.create_history_aware_retriever.html)
 * [MessagesPlaceholder](https://api.python.langchain.com/en/latest/prompts/langchain_core.prompts.chat.MessagesPlaceholder.html)

We can test this out by passing in an instance where the user asks a follow-up question.

In [25]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

[Document(page_content="Every playground run is logged in the system and can be used to create test cases or compare with other runs.Beta Testing\u200bBeta testing allows developers to collect more data on how their LLM applications are performing in real-world scenarios. In this phase, it’s important to develop an understanding for the types of inputs the app is performing well or poorly on and how exactly it’s breaking down in those cases. Both feedback collection and run annotation are critical for this workflow. This will help in curation of test cases that can help track regressions/improvements and development of automatic evaluations.Capturing Feedback\u200bWhen launching your application to an initial set of users, it’s important to gather human feedback on the responses it’s producing. This helps draw attention to the most interesting runs and highlight edge cases that are causing problematic responses. LangSmith allows you to attach feedback scores to logged traces (oftentime

API Reference:
 * [HumanMessage](https://api.python.langchain.com/en/latest/messages/langchain_core.messages.human.HumanMessage.html)
 * [AIMessage](https://api.python.langchain.com/en/latest/messages/langchain_core.messages.ai.AIMessage.html)

You should see that this returns documents about testing in LangSmith. This is because the LLM generated a new query, combining the chat history with the follow-up question.

Now that we have this new retriever, we can create a new chain to continue the conversation with these retrieved documents in mind.

In [26]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

We can now test this out end-to-end:

In [27]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'context': [Document(page_content="Every playground run is logged in the system and can be used to create test cases or compare with other runs.Beta Testing\u200bBeta testing allows developers to collect more data on how their LLM applications are performing in real-world scenarios. In this phase, it’s important to develop an understanding for the types of inputs the app is performing well or poorly on and how exactly it’s breaking down in those cases. Both feedback collection and run annotation are critical for this workflow. This will help in curation of test cases that can help track regressions/improvements and development of automatic evaluations.Capturing Feedback\u200bWhen launching your application to an initial set of users, it’s important to gather human feedback on the responses it’s producing. This helps draw attention to the most i

We can see that this gives a coherent answer - we've successfully turned our retrieval chain into a chatbot!

# Agent

We've so far created examples of chains - where each step is known ahead of time. The final thing we will create is an agent - where the LLM decides what steps to take.

**NOTE: for this example we will only show how to create an agent using OpenAI models, as local models are not reliable enough yet.**

One of the first things to do when building an agent is to decide what tools it should have access to. For this example, we will give the agent access to two tools:

1. The retriever we just created. This will let it easily answer questions about LangSmith
1. A search tool. This will let it easily answer questions that require up-to-date information.

First, let's set up a tool for the retriever we just created:

In [28]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

API Reference:
* [create_retriever_tool](https://api.python.langchain.com/en/latest/tools/langchain_core.tools.create_retriever_tool.html)

The search tool that we will use is [Tavily](https://python.langchain.com/docs/integrations/retrievers/tavily/). This will require an API key (they have generous free tier). After creating it on their platform, you need to set it as an environment variable:

```bash
export TAVILY_API_KEY=...
```

If you do not want to set up an API key, you can skip creating this tool.

In [30]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults()

API Reference:
* [TavilySearchResults](https://api.python.langchain.com/en/latest/tools/langchain_community.tools.tavily_search.tool.TavilySearchResults.html)